# import Libraries

In [1]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 45.4 MB/s eta 0:00:0000:0100:01


In [1]:
# data manipulation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from rapidfuzz import process, fuzz
from sklearn.impute import KNNImputer
import pandas as pd
import io
import warnings
import ast
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold
from category_encoders import TargetEncoder

# data processing and modeling

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import joblib

ModuleNotFoundError: No module named 'rapidfuzz'

# upload Data

In [ ]:
data_1 = pd.read_csv('/kaggle/input/cat-datathon-1-0/Train/train_subset_1.csv')
data_2 = pd.read_csv('/kaggle/input/cat-datathon-1-0/Train/train_subset_2.csv')
data_3 = pd.read_csv('/kaggle/input/cat-datathon-1-0/Train/train_subset_3.csv')
data_4 = pd.read_csv('/kaggle/input/cat-datathon-1-0/Train/train_subset_4.csv')
test = pd.read_csv('/kaggle/input/cat-datathon-1-0/Test/test.csv')
pd.set_option('display.max_columns', None)

In [ ]:
data_1.info()

In [ ]:
data_2.info()

In [ ]:
data_3.info()

In [ ]:
data_4.info()

# combine data

In [ ]:
dfs = [data_1, data_2, data_3, data_4]
for i ,df in enumerate(dfs):
    print(f'DataSet {i+1}:\n', df.columns)

In [ ]:
def clean_column_name(name: str) -> str:
    # 1. First remove unwanted words
    for word in ['field', 'metric', 'var', 'data', 'info', 'measure', 'min', '0', '.', '?']:
        name = name.replace(word, '')   # remove anywhere

    # 2. Convert camelCase → snake_case
    s1 = re.sub(r'(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', s1)

    # 3. Replace spaces/hyphens with underscores
    s3 = re.sub(r'[\s\-]+', '_', s2)

    # 4. Normalize underscores
    s4 = re.sub(r'__+', '_', s3).strip('_')

    # 5. Lowercase
    col = s4.lower()

    # ---- Minimal mapping for ambiguous cases ----
    mapping = {
        'unnamed:': 'id',
        'sc': 'store_cost',
        'ga': 'grocery_area',
        'pn': 'promotion_name',
        'sk': 'store_kind',
        'cus56': 'customer_state',
        'rev54': 'review_score',
    }

    return mapping.get(col, col)


In [ ]:
dfs = [data_1, data_2, data_4]
for df in dfs:
    df.columns = [clean_column_name(col) for col in df.columns]

In [ ]:
dfs = [data_1, data_2, data_3, data_4]
for df in dfs:
    print(df.columns)
    print(len(df.columns))

In [ ]:
data_3.head(2)

In [ ]:
data_4.head(2)

In [ ]:
data_3.columns

In [ ]:
new_columns = [
    'id', 'person_description', 'yearly_income', 'customer_order',
    'gross_weight', 'net_weight', 'unknown', 'is_recyclable',
    'additional_features_in_market', 'promotion_name', 'store_kind',
    'store_sales', 'store_cost', 'grocery_area', 'store_area',
    'frozen_area', 'meat_area', 'cost', 'customer_city', 'customer_state',
    'customer_lat', 'customer_lng', 'seller_city', 'seller_state',
    'seller_lat', 'seller_lng', 'review_score'
]
data_3.columns = new_columns

In [ ]:
data_3.rename(columns={'unknown':'package_weight'}, inplace=True)

In [ ]:
dfs = [data_1, data_2, data_3, data_4]
for i ,df in enumerate(dfs):
    print(f'DataSet {i+1}:\n', df.columns)
    print('Number os columns:', len(df.columns),'\n')


 **before combine :**
- dataset 1 missing store area column
- dataset 2 combine gross weight and net weight
- all data sets except 3 missing package weight

In [ ]:
data_2['weights']

In [ ]:
# clean unwanted text
data_2['weights'] = data_2['weights'].str.replace(r' val:| data:', '', regex=True)

# extract gross and net numbers
data_2['gross_weight'] = data_2['weights'].str.extract(r"'gross':\s*([\d\.]+)")
data_2['net_weight']   = data_2['weights'].str.extract(r"'net':\s*([\d\.]+)")

# drop weights
data_2.drop(columns=['weights'], inplace=True)

- after cleaning the columns will be able to exctract store area and package wiat from the other columns

In [ ]:
train = pd.concat([data_1, data_2, data_3, data_4], ignore_index=True)

In [ ]:
train.head()

In [ ]:
test.columns

In [ ]:
test.columns = [clean_column_name(col) for col in test.columns]

In [ ]:
train.info()

In [ ]:
train.apply(lambda col: pd.Series({"type": str(col.dtype),
                                "null percentage": round(col.isna().sum() / len(train) * 100, 2),
                                'nunique': col.nunique(),
                                'unique': col.unique().tolist()})).T

- address some issues found in almost every column

In [ ]:
missing_vals = [
    'unknown', 'data missing', 'to be determined', 'nil', '...', '-1', '-999',
    '-1.0', '-999.0', 'not available', 'tbd', '?', 'missing', '--',
    'not specified', '###',
]

def clean_unwanted_string(df):
    df = df.copy()

    for col in df.columns:
        if df[col].dtype == "object":
            # standardize text
            df[col] = df[col].str.strip().str.lower()
            # replace missing tokens
            df[col] = df[col].replace(missing_vals, np.nan)

    return df

In [ ]:
train = clean_unwanted_string(train)

In [ ]:
test = clean_unwanted_string(test)

- check duplicates

In [ ]:
train.duplicated().sum()

In [ ]:
train.drop_duplicates(inplace=True)

# Data cleaning

## id colomn

In [ ]:
train['id'].unique()

In [ ]:
train['id'].isna().sum()

In [ ]:
train['id'] = train['id'].str.extract(r'(\d+\.?\d*)')

In [ ]:
train['id'] = range(len(train))

### summary
- because it's conmbine from multiple datasets the column has duplicated values
- for now I just assigned all values form 0 to data lenght
- now no missing values or duplicates

## person_description column

### clean

In [ ]:
train['person_description'].unique()[:20]

In [ ]:
train['person_description'].isna().sum()

- select (male/female)
- select (single/married)
- select (numbers of kid/kids/child/childern)
- select education
- select work

In [ ]:
import difflib

word_to_num = {
    "zero": 0, "no": 0, "none": 0,
    "one": 1, "two": 2, "three": 3, "four": 4, "five": 5
}

status_map = {
    "single": ["single", "singlee", "singel"],
    "married": ["married", "maried", "marrid"]
}

education_map = {
    "bachelors": ["bachelors", "batchelors", "bachelos", "bachelor'ss"],
    "high school": ["high school", "hs", "highschool", 'high  school'],
    "partial college": ["partial college", "partial collge", "partial colllege"],
    "partial high school": ["partial high school", "partial hs"],
    "graduate": ["graduate", "gradute"]
}

gender_map = {
    "male": ["male"],
    "female": ["female", "femal"]
}

occupation_map = {
    "professional": ["professional"],
    "management": ["management"],
    "clerical": ["clerical"],
    "manual": ["manual"],
    "skilled manual": ["skilled manual", "skilled"],
    "unskilled manual": ["unskilled manual", "unskilled"]
}


def normalize(text):
    """Lowercase and remove special chars."""
    return re.sub(r"[^a-z\s]", " ", str(text).lower()).strip()

def extract_exact(text, mapping):
    """Return the matched canonical key if any variant matches exactly as word."""
    txt = normalize(text).split()
    for key, variants in mapping.items():
        for v in variants:
            if v in txt:
                return key
    return np.nan

def extract_status(text, cutoff=0.8):
    """Fuzzy match status (single/married) even with typos."""
    txt = normalize(text)
    words = txt.split()

    # Flatten variants
    all_variants = {v: key for key, vals in status_map.items() for v in vals}

    for w in words:
        match = difflib.get_close_matches(w, all_variants.keys(), n=1, cutoff=cutoff)
        if match:
            return all_variants[match[0]]

    return np.nan

def extract_children(text):
    """Return number of children (0–5 max)."""
    txt = normalize(text).split()
    for word, num in word_to_num.items():
        if word in txt:
            return min(num, 5)
    return np.nan

def extract_education(text, cutoff=0.8):
    """Match education (exact phrase or fuzzy word match)."""
    txt = normalize(text)

    # Flatten map
    all_variants = {v: key for key, vals in education_map.items() for v in vals}

    # 1) Exact substring (works for multi-word like 'high school')
    for v, key in all_variants.items():
        if v in txt:
            return key

    # 2) Fuzzy word match (for typos like 'bachelos', 'gradute')
    words = txt.split()
    for w in words:
        match = difflib.get_close_matches(w, all_variants.keys(), n=1, cutoff=cutoff)
        if match:
            return all_variants[match[0]]

    return np.nan

# main function

def extract_person_description(df, col="person_description"):
    df["status"] = df[col].apply(extract_status)
    df["gender"] = df[col].apply(lambda x: extract_exact(x, gender_map))
    df["children"] = df[col].apply(extract_children)
    df["education"] = df[col].apply(extract_education)
    df["work"] = df[col].apply(lambda x: extract_exact(x, occupation_map))
    return df

In [ ]:
train = extract_person_description(train)
train.drop(columns=["person_description"], inplace=True)

In [ ]:
test = extract_person_description(test)
test.drop(columns=["person_description"], inplace=True)

In [ ]:
cols = ["gender", "status", "children", "education", "work"]

print(train[cols].isnull().sum())

### impute

In [ ]:
cols = ["gender", "status", "children", "education", "work"]

for col in cols:
    if train[col].isnull().any():
        mode_val = train[col].mode().iloc[0]  
        train[col].fillna(mode_val, inplace=True)

In [ ]:
# الأعمدة المستهدفة
cols = ["gender", "status", "children", "education", "work"]

# نحتفظ بقيم mode لكل عمود من train
fill_values = {}

for col in cols:
    if train[col].isnull().any():
        mode_val = train[col].mode().iloc[0]
        train[col].fillna(mode_val, inplace=True)
        fill_values[col] = mode_val
    else:
        # حتى لو مفيش NaN نحفظ قيمة mode
        fill_values[col] = train[col].mode().iloc[0]

# نطبق نفس القيم على test
for col in cols:
    test[col].fillna(fill_values[col], inplace=True)


### Summary
- New features extracted (gender, status, children, education, work)
- impute those features with the mode
- dropped the original column

## customer_order column

In [ ]:
print(train["customer_order"].unique()[:20])

* select product
* select department
* select brand

### clean

In [ ]:
def extract_order(order):
    if pd.isna(order):
        return pd.Series([np.nan, np.nan, np.nan])

    order = str(order).lower().strip()
    order = order.replace("(", "").replace(")", "").replace(".", "")

    product, department, brand = np.nan, np.nan, np.nan

    # Extract product
    prod_match = re.search(r"^([a-z\s]+?)\s+from", order)
    if prod_match:
        product = prod_match.group(1).strip()

    # Extract department
    dept_match = re.search(r"from\s+(.+?)\s+department", order)
    if dept_match:
        department = dept_match.group(1).strip()

    # Extract brand
    brand_match = re.search(r"ordered brand\s*:\s*([a-z\s\-]+)", order)
    if brand_match:
        brand = brand_match.group(1).strip()

    return pd.Series([product, department, brand])

In [ ]:
# تطبيق الدالة على train
train[["product", "department", "brand"]] = train["customer_order"].apply(extract_order)
train.drop(columns=["customer_order"], inplace=True)

# تطبيق نفس الدالة على test
test[["product", "department", "brand"]] = test["customer_order"].apply(extract_order)
test.drop(columns=["customer_order"], inplace=True)


### impute

In [ ]:
cols = ["product", "department", "brand"]

for col in cols:
    print(f"Number of NaN in {col} befor fillna:", train[col].isna().sum())

In [ ]:
cols = ["product", "department", "brand"]

for col in cols:
    # حساب المود
    mode_val = train[col].mode()[0]  # نأخذ أول مود في حال وجود أكثر من مود
    # استبدال القيم المفقودة بالمود
    train[col].fillna(mode_val, inplace=True)

In [ ]:
cols = ["product", "department", "brand"]

# نخزن قيم mode من train
fill_values = {}

for col in cols:
    mode_val = train[col].mode()[0]
    train[col].fillna(mode_val, inplace=True)
    fill_values[col] = mode_val  # نخزنها لاستخدامها مع test

# نطبق نفس القيم على test
for col in cols:
    test[col].fillna(fill_values[col], inplace=True)

# التحقق
for col in cols:
    print(f"{col} - NaN in train:", train[col].isna().sum(),
          "| NaN in test:", test[col].isna().sum())


### Summary
- New features extracted (product, department, brand)
- imputed those features with the mode
- dropped the original column

## yearly_income column

### clean

In [ ]:
train['yearly_income'].unique()

**problems:**
- monthly >> annually
- k, Thousand >> 1000
- € >> $
- special characters (select only numbers)

In [ ]:
def clean_income(col):
    # Masks
    monthly_mask = col.str.contains('monthly', case=False, na=False)
    euro_mask = col.str.contains("€", na=False)
    k_mask = col.str.contains(r'k|thousand', case=False, na=False)

    # Extract numbers (including decimals)
    numbers = col.str.extract(r'(\d+\.?\d*)')
    numbers = pd.to_numeric(numbers[0], errors='coerce')

    # Apply transformations
    numbers.loc[k_mask] *= 1000
    numbers.loc[monthly_mask] *= 12
    numbers.loc[euro_mask] *= 1.1  # تحويل يورو لدولار تقريبي

    # رجع Int لو الرقم صحيح، Float لو فيه كسور
    numbers = numbers.apply(lambda x: int(x) if pd.notna(x) and x.is_integer() else x)

    return numbers

In [ ]:
train['yearly_income'] = clean_income(train['yearly_income'])

In [ ]:
test['yearly_income'] = clean_income(test['yearly_income'])

### outliers

In [ ]:
train['yearly_income'].unique()

In [ ]:
print(train['yearly_income'].describe())

# Outliers detection
Q1 = train['yearly_income'].quantile(0.25)
Q3 = train['yearly_income'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = train[(train['yearly_income'] < lower_bound) | (train['yearly_income'] > upper_bound)]
print(f"\nNumber of outliers: {outliers.shape[0]}")
print("Sample outliers:")
print(outliers['yearly_income'].head())

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
train['yearly_income'].hist(bins=50, edgecolor='black')
plt.title("Distribution of Yearly Income")
plt.xlabel("Yearly Income")
plt.ylabel("Frequency")

plt.subplot(1,2,2)
train.boxplot(column='yearly_income')
plt.title("Boxplot of Yearly Income")

plt.tight_layout()
plt.show()

Given the data from Brazil:

The average annual income in Brazil is much lower (around $8,000–$12,000 per year for the majority).

Even the upper classes rarely reach $150,000 per year.

Therefore, the value of $150,000 in this case is clearly an outlier.

In [ ]:
threshold = 140000

# تحويل outliers لـ NaN
train.loc[train['yearly_income'] > threshold, 'yearly_income'] = np.nan

In [ ]:
test.loc[test['yearly_income'] > threshold, 'yearly_income'] = np.nan

### impute

In [ ]:
train['yearly_income'].isna().sum()

In [ ]:
train['yearly_income'] = train['yearly_income'].fillna(train['yearly_income'].median())

In [ ]:
# حساب median من train
median_income = train['yearly_income'].median()

# تعويض القيم المفقودة في train
train['yearly_income'] = train['yearly_income'].fillna(median_income)

# تعويض القيم المفقودة في test بنفس median من train
test['yearly_income'] = test['yearly_income'].fillna(median_income)


In [ ]:
train['yearly_income'].isna().sum()

-  imputed with median
-  median is the most appropriate method here

### Summary
- handled data inconsistency
- anything above certain threshold = 140000 is considered outliers
- missing values and outliers were imputed with the median

## is_recyclable column

### clean

In [ ]:
train['is_recyclable'].unique()

In [ ]:
def clean_recyclable(df, col="is_recyclable"):
    df[col] = df[col].str.replace(r'[().]', '', regex=True)

    df[col] = df[col].replace({'recyclable': 'yes','non recyclable': 'no'})
    valid = ['yes', 'no']
    df.loc[~df[col].isin(valid), col] = np.nan

    return df[col]

In [ ]:
train['is_recyclable'] = clean_recyclable(train, 'is_recyclable')

In [ ]:
test['is_recyclable'] = clean_recyclable(test, 'is_recyclable')

In [ ]:
train['is_recyclable'].value_counts()

In [ ]:
train['is_recyclable'].isna().sum()

### impute

In [ ]:
# impute with random values to keep the original ratio of the classes

def random_impute(series):
    series = series.copy()
    missing = series.isna()
    series.loc[missing] = np.random.choice(series.dropna(), size=missing.sum(), replace=True)
    return series

In [ ]:
train['is_recyclable'] = random_impute(train['is_recyclable'])

In [ ]:
# دالة التعويض العشوائي باستخدام قيم train فقط
def random_impute_from_train(series, train_series):
    series = series.copy()
    missing = series.isna()
    if missing.sum() > 0:
        series.loc[missing] = np.random.choice(train_series.dropna(),
                                               size=missing.sum(),
                                               replace=True)
    return series

# تعويض في train
train['is_recyclable'] = random_impute(train['is_recyclable'])

# تعويض في test باستخدام توزيع train
test['is_recyclable'] = random_impute_from_train(test['is_recyclable'], train['is_recyclable'])


In [ ]:
train['is_recyclable'].isna().sum()

### Summary
- set the unique values to yes/no
- imputed with random value to keep the original ratio of the classes

## store_kind column

### clean

In [ ]:
train['store_kind'].nunique()

In [ ]:
train['store_kind'].unique()[:50]

- Match the values to unique values

In [ ]:
def clean_store_kind(df, col):
    
    from thefuzz import process
    kinds = ['deluxe', 'supermarket', 'gourmet', 'mid size', 'small grocery']

    for kind in kinds:
        matches = process.extract(kind, df[col].unique(), limit= len(df[col].unique()))
        for i in matches:
            if i[1] >= 80:
                df.loc[df[col] == i[0], col] = kind
                
    return df[col]

In [ ]:
train['store_kind'] = clean_store_kind(train,'store_kind')

In [ ]:
train['store_kind'].value_counts()

In [ ]:
train['store_kind'].isna().sum()

In [ ]:
test['store_kind'] = clean_store_kind(test,'store_kind')

### impute

Best methods:
- If the deficiency is slight, we compensate with moods.
- If the deficiency is moderate, we make it unknown.
- We can also make a KNN impute , but it's not the best option here.

In [ ]:
# استبدال القيم المفقودة في store_kind بـ "unknown"
train["store_kind"] = train["store_kind"].fillna("unknown")

# نطبع توزيع القيم بعد التعديل
print(train["store_kind"].value_counts(dropna=False))


In [ ]:
# تعويض القيم المفقودة في test بنفس القيمة الثابتة
test["store_kind"] = test["store_kind"].fillna("unknown")

In [ ]:
train['store_kind'].isna().sum()

In [ ]:
# نعمل mapping للقيم
mapping = {
    "supermarket": "supermarket",
    "mid size": "supermarket",
    "deluxe": "premium",
    "gourmet": "premium",
    "small grocery": "small",
    "unknown": "unknown"
}

# نطبق المابنج
train["store_kind"] = train["store_kind"].map(mapping)

# نتأكد من التوزيع بعد الدمج
print(train["store_kind"].value_counts())


In [ ]:
test["store_kind"] = test["store_kind"].map(mapping)

### Summary
- match the values to unique values with fuzzy matching
- set the missing values to be "unknown"


## customer_city column

### clean

In [ ]:
train[['customer_city','seller_city']]

In [ ]:
train[['customer_city', 'seller_city']].nunique()

In [ ]:
train['customer_city'].value_counts().head(20)

In [ ]:
# Keep only letters and spaces, remove numbers and symbols
train['customer_city'] = train['customer_city'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Remove extra spaces if any
train['customer_city'] = train['customer_city'].str.strip()
train['customer_city'] = train['customer_city'].str.replace(r'\s+', ' ', regex=True)


In [ ]:
# Keep only letters and spaces, remove numbers and symbols
test['customer_city'] = test['customer_city'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Remove extra spaces if any
test['customer_city'] = test['customer_city'].str.strip()
test['customer_city'] = test['customer_city'].str.replace(r'\s+', ' ', regex=True)


In [ ]:
# Count null values in customer_city
null_count = train['customer_city'].isnull().sum()

print("Number of null values in customer_city:", null_count)


### impute

In [ ]:
# impute with random values to keep the original ratio of the classes
train['customer_city'] = random_impute(train['customer_city'])

In [ ]:
# Impute in test set with random sampling from existing values
test['customer_city'] = random_impute(test['customer_city'])


In [ ]:
# Count null values in customer_city
null_count = train['customer_city'].isnull().sum()

print("Number of null values in customer_city:", null_count)

In [ ]:
train['customer_city'].nunique()

In [ ]:
# تحويل القيم لنصوص والتعامل مع القيم الفارغة
train['customer_city'] = train['customer_city'].fillna("").astype(str)

unique_seller_cities = train['customer_city'].unique()
THRESHOLD = 85  # نفس الفكرة من الـ customer_city

# القيم المرجعية: أكتر المدن تكرارًا (optional: لو عندك قائمة محددة)
# هنا نستخدم كل القيم الفريدة عشان نقارن بيها
seller_city_map = {}

for city in unique_seller_cities:
    if city in seller_city_map:
        continue
    matches = process.extract(city, unique_seller_cities, scorer=fuzz.ratio, limit=None)
    for match_tuple in matches:
        if len(match_tuple) == 3:
            match, score, _ = match_tuple
        else:
            match, score = match_tuple
        if score >= THRESHOLD:
            seller_city_map[match] = city

# تطبيق الخريطة باستخدام get عشان القيم اللي مش موجودة تفضل كما هي
train['customer_city'] = train['customer_city'].apply(lambda x: seller_city_map.get(x, x))

# عرض عدد القيم الفريدة بعد التنظيف
print("Unique seller cities after cleaning:", train['customer_city'].nunique())


In [ ]:
# تحويل القيم لنصوص والتعامل مع القيم الفارغة
test['customer_city'] = test['customer_city'].fillna("").astype(str)

# تطبيق نفس الخريطة المستخرجة من train
test['customer_city'] = test['customer_city'].apply(lambda x: seller_city_map.get(x, x))



In [ ]:
# Get the 2500 least frequent values
least_2500 = train['customer_city'].value_counts().sort_values().head(2400)

print(least_2500)


**More than 2400 cities occur less than 10 times, which is a very small frequency, so they were replaced with 'other'.**



In [ ]:

# Count occurrences of each city
city_counts = train['customer_city'].value_counts()

# Find cities with less than 10 occurrences
rare_cities = city_counts[city_counts < 10].index

# Replace them with 'other'
train['customer_city'] = train['customer_city'].replace(rare_cities, 'other')


In [ ]:
# Count occurrences of each city in train
city_counts = train['customer_city'].value_counts()

# Find rare cities (less than 10 occurrences in train)
rare_cities = city_counts[city_counts < 10].index

# Replace them with 'other' in train
train['customer_city'] = train['customer_city'].replace(rare_cities, 'other')

# Apply the SAME mapping on test (without recalculating)
test['customer_city'] = test['customer_city'].replace(rare_cities, 'other')



In [ ]:
 print("Unique values:", train['customer_city'].nunique())

This is the lowest number I can reach

In [ ]:
train['customer_city'].value_counts().head(20)

- Removing these columns is not a good idea at all, as the location has an impact on the cost (transportation costs/taxes/location).
- I will do a targeted/freq encoding for this column.

### Summary
- match the cities with fuzzy matching
- set the least frequent cities (less than 10 records) to be "other"
- imputed with random chioce as is_recyclable

## seller_city column

### clean

In [ ]:
train['seller_city'].nunique()

In [ ]:
# Keep only letters and spaces, remove numbers and symbols
train['seller_city'] = train['seller_city'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Remove extra spaces if any
train['seller_city'] = train['seller_city'].str.strip()
train['seller_city'] = train['seller_city'].str.replace(r'\s+', ' ', regex=True)

In [ ]:
# Keep only letters and spaces, remove numbers and symbols
test['seller_city'] = test['seller_city'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Remove extra spaces if any
test['seller_city'] = test['seller_city'].str.strip()
test['seller_city'] = test['seller_city'].str.replace(r'\s+', ' ', regex=True)


In [ ]:
train['seller_city'].nunique()

In [ ]:
# Count null values in seller_city
null_count = train['seller_city'].isnull().sum()

print("Number of null values in seller_city:", null_count)

In [ ]:
# impute with random values to keep the original ratio of the classes
train['seller_city'] = random_impute(train['seller_city'])

In [ ]:
import numpy as np

# indices اللي فيها NaN
missing_idx = test['seller_city'].isna()

# تعويض القيم المفقودة فقط
test.loc[missing_idx, 'seller_city'] = np.random.choice(
    train['seller_city'].dropna(),
    size=missing_idx.sum(),
    replace=True
)


In [ ]:
train['seller_city'].isnull().sum()

### impute

In [ ]:
# تحويل القيم لنصوص والتعامل مع القيم الفارغة
train['seller_city'] = train['seller_city'].fillna("").astype(str)

unique_seller_cities = train['seller_city'].unique()
THRESHOLD = 85  # نفس الفكرة من الـ customer_city

# القيم المرجعية: أكتر المدن تكرارًا (optional: لو عندك قائمة محددة)
# هنا نستخدم كل القيم الفريدة عشان نقارن بيها
seller_city_map = {}

for city in unique_seller_cities:
    if city in seller_city_map:
        continue
    matches = process.extract(city, unique_seller_cities, scorer=fuzz.ratio, limit=None)
    for match_tuple in matches:
        if len(match_tuple) == 3:
            match, score, _ = match_tuple
        else:
            match, score = match_tuple
        if score >= THRESHOLD:
            seller_city_map[match] = city

# تطبيق الخريطة باستخدام get عشان القيم اللي مش موجودة تفضل كما هي
train['seller_city'] = train['seller_city'].apply(lambda x: seller_city_map.get(x, x))

# عرض عدد القيم الفريدة بعد التنظيف
print("Unique seller cities after cleaning:", train['seller_city'].nunique())



In [ ]:
# تحويل القيم لنصوص والتعامل مع القيم الفارغة
test['seller_city'] = test['seller_city'].fillna("").astype(str)

# تطبيق نفس الخريطة المستخرجة من train
test['seller_city'] = test['seller_city'].apply(lambda x: seller_city_map.get(x, x))

In [ ]:
train['seller_city'].value_counts().head(20)

In [ ]:
# Get the 500 least frequent values
least_500 = train['seller_city'].value_counts().sort_values().head(500)

print(least_500)


In [ ]:
# Count occurrences of each city
s_city_counts = train['seller_city'].value_counts()

# Find cities with less than 10 occurrences
s_rare_cities = s_city_counts[s_city_counts < 10].index

# Replace them with 'other'
train['seller_city'] = train['seller_city'].replace(s_rare_cities, 'other')

In [ ]:
# Count occurrences of each city in train
s_city_counts = train['seller_city'].value_counts()

# Find rare cities (less than 10 occurrences in train)
s_rare_cities = s_city_counts[s_city_counts < 10].index

# Replace them with 'other' in train
train['seller_city'] = train['seller_city'].replace(s_rare_cities, 'other')

# Apply the SAME mapping on test (without recalculating)
test['seller_city'] = test['seller_city'].replace(s_rare_cities, 'other')



In [ ]:
print("Unique values:", train['customer_city'].nunique())

In [ ]:
# نجيب متوسط الكوست لكل مدينة
#city_encoding = train.groupby("seller_city")["cost"].mean()

# نعمل مابنج للعمود
#train["seller_city_encoded"] = train["seller_city"].map(city_encoding)

# نعمل راوند لـ 3 أرقام عشرية
#train["seller_city_encoded"] = train["seller_city_encoded"].round(3)



In [ ]:

# لو عايزة تشيلي العمود الأصلي
# train = train.drop(columns=["seller_city"])


In [ ]:
#train.head()

### summary
- same process as customer_city

## customer_state column

### clean

In [ ]:
train[['customer_state','seller_state']].nunique()

In [ ]:
train[['customer_state','seller_state']] = train[['customer_state','seller_state']].apply(lambda col: col.str.upper().str.strip())
train[['customer_state','seller_state']] = train[['customer_state','seller_state']].apply(lambda col: col.str.replace(r'[()._]', '', regex=True).str.strip())

In [ ]:
# تحويل القيم لحروف كبيرة وإزالة المسافات الزائدة
test[['customer_state','seller_state']] = test[['customer_state','seller_state']].apply(lambda col: col.str.upper().str.strip())

# إزالة الرموز [()._] وإعادة trim
test[['customer_state','seller_state']] = test[['customer_state','seller_state']].apply(lambda col: col.str.replace(r'[()._]', '', regex=True).str.strip())

In [ ]:
train[['customer_state','seller_state']].nunique()

In [ ]:
train['customer_state'].value_counts().head(10)

In [ ]:
train['customer_state'].unique()

### impute

In [ ]:
train['customer_state'].isna().sum()

In [ ]:
# دكشنري للتحويلات الرمزية
replace_dict = {
    'Ñ': 'N',
    'Ç': 'C',
    '@': 'A',
    '$': 'S',
    '5': 'S',
    '3': 'E',
    '0': 'O',
    '4': 'A',
    '7': 'T',
    '1': 'L',
}

# دالة التطبيع الشامل
def normalize_state(val):
    if pd.isna(val):
        return np.nan
    val = str(val).upper().strip()             # Uppercase و إزالة المسافات
    if val in ['', 'NAN']:
        return np.nan
    val = val.replace('_', '')                 # إزالة _
    val = val.replace('\n', '')                # إزالة أسطر جديدة
    val = re.sub(r'[\.\?\(\)]', '', val)      # إزالة . ? ( )
    val = val.replace('--', '')                # إزالة --
    val = val.replace('...', '')               # إزالة ...
    for old, new in replace_dict.items():      # استبدال الرموز والحروف الخاصة
        val = val.replace(old, new)
    return val

# تطبيق التطبيع
train['customer_state'] = train['customer_state'].apply(normalize_state)

# حساب المود بعد التطبيع
mode_value = train['customer_state'].mode()[0]

# استبدال NaN بالمود
train['customer_state'] = train['customer_state'].fillna(mode_value)

# عرض القيم الفريدة بعد كل التنظيف والتعبئة
print(train['customer_state'].unique())


In [ ]:
# تطبيق التطبيع على test
test['customer_state'] = test['customer_state'].apply(normalize_state)

# استبدال NaN بنفس المود من train
test['customer_state'] = test['customer_state'].fillna(mode_value)



In [ ]:
train['customer_state'].nunique()

In [ ]:
train['customer_state'].isna().sum()

In [ ]:
# Get the 10 least frequent values
least_10 = train['customer_state'].value_counts().sort_values().head(10)

print(least_10)


**about 10 states occur less than 10 times, which is a very small frequency, so they were replaced with 'other'.**

In [ ]:
# Count occurrences of each city
c_city_counts = train['customer_state'].value_counts()

# Find cities with less than 10 occurrences
c_rare_cities = c_city_counts[c_city_counts < 10].index

# Replace them with 'other'
train['customer_state'] = train['customer_state'].replace(c_rare_cities, 'other')

In [ ]:
# Count occurrences of each state in train
c_city_counts = train['customer_state'].value_counts()

# Find rare states (less than 10 occurrences in train)
c_rare_cities = c_city_counts[c_city_counts < 10].index

# Replace them with 'other' in train
train['customer_state'] = train['customer_state'].replace(c_rare_cities, 'other')

# Apply the SAME mapping on test (without recalculating)
test['customer_state'] = test['customer_state'].replace(c_rare_cities, 'other')

In [ ]:
print("Unique values:", train['customer_state'].nunique())

### Summary
- mapped special characters to the original characters
- imputed with the mode
- set the least frequent states (less than 10 records) to be "other"

## seller_state column

### clean & impute

In [ ]:
train['seller_state'].unique()[:20]

In [ ]:
# دكشنري للتحويلات الرمزية والأرقام للحروف المكافئة
replace_dict = {
    'Ñ': 'N',
    'Ç': 'C',
    '@': 'A',
    '$': 'S',
    '5': 'S',
    '3': 'E',
    '0': 'O',
    '4': 'A',
    '7': 'T',
    '1': 'L',
}

# دالة التطبيع الشامل
def normalize_state(val):
    if pd.isna(val):
        return np.nan
    val = str(val).upper().strip()             # Uppercase و إزالة المسافات
    if val in ['', 'NAN']:
        return np.nan
    val = val.replace('_', '')                 # إزالة _
    val = val.replace('\n', '')                # إزالة أسطر جديدة
    val = re.sub(r'[\.\?\(\)]', '', val)      # إزالة . ? ( )
    val = val.replace('--', '')                # إزالة --
    val = val.replace('...', '')               # إزالة ...
    for old, new in replace_dict.items():      # استبدال الرموز والأرقام
        val = val.replace(old, new)
    return val

# تطبيق التطبيع
train['seller_state'] = train['seller_state'].apply(normalize_state)

# حساب المود بعد التطبيع
mode_value = train['seller_state'].mode()[0]

# استبدال NaN بالمود
train['seller_state'] = train['seller_state'].fillna(mode_value)

# عرض القيم الفريدة بعد كل التنظيف والتعبئة
print(train['seller_state'].unique())


In [ ]:
# تطبيق التطبيع على test
test['seller_state'] = test['seller_state'].apply(normalize_state)

# استبدال NaN بنفس المود من train
test['seller_state'] = test['seller_state'].fillna(mode_value)



In [ ]:
# Get the 10 least frequent values
least_10 = train['seller_state'].value_counts().sort_values().head(10)

print(least_10)

**about 10 states occur less than 10 times, which is a very small frequency, so they were replaced with 'other'.**

In [ ]:
# Count occurrences of each city
c_city_states = train['seller_state'].value_counts()

# Find cities with less than 10 occurrences
c_rare_states = c_city_states[c_city_states < 10].index

# Replace them with 'other'
train['seller_state'] = train['seller_state'].replace(c_rare_states, 'other')

In [ ]:
# Count occurrences of each state in train
c_city_states = train['seller_state'].value_counts()

# Find rare states (less than 10 occurrences in train)
c_rare_states = c_city_states[c_city_states < 10].index

# Replace them with 'other' in train
train['seller_state'] = train['seller_state'].replace(c_rare_states, 'other')

# Apply the SAME mapping on test (without recalculating)
test['seller_state'] = test['seller_state'].replace(c_rare_states, 'other')



In [ ]:
print("Unique values:", train['seller_state'].nunique())

### Summary
- same process as customer_state

## customer_lat, customer_lng, seller_lat, seller_lng columns

In [ ]:
geo_columns = ['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng']

for col in geo_columns:
    unique_values = train[col].dropna().unique()
    print(f"\n--- {col.upper()} ---")
    print(f"Number of unique values: {len(unique_values)}")
    print("Sample unique values:", unique_values[:20])  # يعرض أول 20 قيمة فقط


### clean

In [ ]:
geo_columns = ['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng']

def extract_float(val):
    if pd.isna(val):
        return np.nan
    # البحث عن الرقم العشري داخل النص
    match = re.search(r"-?\d+\.\d+", str(val))
    if match:
        return float(match.group())
    return np.nan

# تطبيق الدالة على كل الأعمدة الجغرافية
for col in geo_columns:
    train[col] = train[col].apply(extract_float)

In [ ]:
for col in geo_columns:
    test[col] = test[col].apply(extract_float)

In [ ]:
geo_columns = ['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng']

for col in geo_columns:
    unique_values = train[col].dropna().unique()
    print(f"\n--- {col.upper()} ---")
    print(f"Number of unique values: {len(unique_values)}")
    print("Sample unique values:", unique_values[:20])  # يعرض أول 20 قيمة فقط

In [ ]:
geo_columns = ['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng']

print("Null values:")
for col in geo_columns:
    print(f"{col}: {train[col].isna().sum()}")


# التحقق من النطاقات الصحيحة
train.loc[~train['customer_lat'].between(-90, 90), 'customer_lat'] = np.nan
train.loc[~train['seller_lat'].between(-90, 90), 'seller_lat'] = np.nan
train.loc[~train['customer_lng'].between(-180, 180), 'customer_lng'] = np.nan
train.loc[~train['seller_lng'].between(-180, 180), 'seller_lng'] = np.nan

# عرض عدد القيم المفقودة بعد التنظيف
print("Null values:")
for col in geo_columns:
    print(f"{col}: {train[col].isna().sum()}")

In [ ]:
geo_columns = ['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng']

print("Null values before cleaning in test:")
for col in geo_columns:
    print(f"{col}: {test[col].isna().sum()}")

# التحقق من النطاقات الصحيحة في test
test.loc[~test['customer_lat'].between(-90, 90), 'customer_lat'] = np.nan
test.loc[~test['seller_lat'].between(-90, 90), 'seller_lat'] = np.nan
test.loc[~test['customer_lng'].between(-180, 180), 'customer_lng'] = np.nan
test.loc[~test['seller_lng'].between(-180, 180), 'seller_lng'] = np.nan

# عرض عدد القيم المفقودة بعد التنظيف
print("Null values after cleaning in test:")
for col in geo_columns:
    print(f"{col}: {test[col].isna().sum()}")


So there is no value outside the correct ranges.

In [ ]:
def haversine_distance(lat1, lng1, lat2, lng2):
    # تحويل الدرجات إلى راديان
    lat1, lng1, lat2, lng2 = map(np.radians, [lat1, lng1, lat2, lng2])

    # الفروق
    dlat = lat2 - lat1
    dlng = lng2 - lng1

    # صيغة Haversine
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlng/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # نصف قطر الأرض بالكيلومتر
    return c * r

# إنشاء عمود جديد للمسافة
train['distance_km'] = haversine_distance(
    train['customer_lat'], train['customer_lng'],
    train['seller_lat'], train['seller_lng']
)

# عرض بعض القيم للتحقق
print(train[['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng', 'distance_km']].head())
# تقريب القيم لثلاثة أرقام عشرية
train['distance_km'] = train['distance_km'].round(3)

In [ ]:
test['distance_km'] = haversine_distance(
    test['customer_lat'], test['customer_lng'],
    test['seller_lat'], test['seller_lng']
)

# تقريب القيم لثلاثة أرقام عشرية
test['distance_km'] = test['distance_km'].round(3)

In [ ]:
train = train.drop(columns=['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng'])

In [ ]:
test = test.drop(columns=['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng'])

In [ ]:
train.head(5)

### outliers

In [ ]:
col = 'distance_km'  # غيره باسم العمود بتاعك

# حساب IQR والحدود
Q1 = train[col].quantile(0.25)
Q3 = train[col].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# فصل القيم الطبيعية عن الأوتلايرز
normal_values = train[(train[col] >= lower_bound) & (train[col] <= upper_bound)][col]
outliers = train[(train[col] < lower_bound) | (train[col] > upper_bound)][col]

# =========================
# 2. Boxplot for outliers
# =========================
plt.figure(figsize=(10,3))
plt.boxplot(train[col], vert=False, patch_artist=True,
            boxprops=dict(facecolor='lightblue'))
plt.title(f'Boxplot of {col} (showing outliers)')
plt.xlabel(col)
plt.show()

print(f"Number of outliers: {len(outliers)}")
print(f"upper_bound: {upper_bound}")


The two most distant points in Brazil, geographically speaking, are:

From the northeast (near the city of Caburaí in Roraima state)

To the south (near the city of Arroio Chuí in Rio Grande do Sul state)

The distance between them is approximately 4,400–4,500 km.

In [ ]:
upper_bound = 1736.955625  # الحد اللي حسبناه

# اختيار الأوتلايرز
outliers_df = train[train['distance_km'] > upper_bound][
    ['customer_city', 'seller_city', 'distance_km']
].sort_values(by='distance_km', ascending=False)

# عرض أول 10 بس
print(outliers_df.head(10))


I found that these extreme values ​​are close to the actual distance between these cities, and anything greater than 4000 is illogical.

In [ ]:
train = train[train['distance_km'] <= 4000]

### Summary
- extract the number for this columns
- extract new feature 'distance_km' the distance between customer and seller location
- dropped the original column
- consider anything above a threshold outliers and removed it

## promotion_name column

### clean

In [ ]:
train['promotion_name'] = train['promotion_name'].str.replace(r'[().]', '', regex=True).str.strip()

In [ ]:
test['promotion_name'] = test['promotion_name'].str.replace(r'[().]', '', regex=True).str.strip()

In [ ]:
train['promotion_name'].unique()

In [ ]:
train['promotion_name'].nunique()

In [ ]:
# عرض كل القيم اليونيك وعدد مرات تكرارها
train['promotion_name'].value_counts()

In [ ]:
train['promotion_name'].isna().sum()

### impute

In [ ]:
# 2. استبدال NaN بـ "No Promo"
train['promotion_name'] = train['promotion_name'].fillna('No Promo')

In [ ]:
test['promotion_name'] = test['promotion_name'].fillna('No Promo')

### Summary
- missing values were set to "No Promo"

## review_score column

In [ ]:
train['review_score'].unique()

### clean

In [ ]:
def clean_score(val):
    if pd.isna(val):
        return np.nan

    s = str(val).lower().strip()
    s = re.sub(r'(val:|measure:|rating:|amount:|data:|stars|/5\.0)', '', s)
    s = re.sub(r'[()]', '', s).strip()

    if s.endswith("%"):
        num = float(s.replace("%", ""))
        return round(num *0.05, 1)

    if "/10" in s:
        num = float(s.split("/")[0])
        return num *0.5

    match = re.match(r"(\d+(\.\d+)?)\.?$", s)
    if match:
        return float(match.group(1))

    return np.nan

train["review_score"] = train["review_score"].apply(clean_score)

In [ ]:
test["review_score"] = test["review_score"].apply(clean_score)

In [ ]:
train["review_score"].isna().sum()

In [ ]:
train["review_score"].value_counts()

### impute

In [ ]:
counts = train['review_score'].value_counts(dropna=True)

choices = counts.index.to_list()
probs = (counts / counts.sum()).to_list()

missing_idx = train[train['review_score'].isna()].index

train.loc[missing_idx, 'review_score'] = np.random.choice(choices, size=len(missing_idx), p=probs)

In [ ]:
# القيم الممكنة واحتمالاتها من train
counts = train['review_score'].value_counts(dropna=True)
choices = counts.index.to_list()
probs = (counts / counts.sum()).to_list()

# العثور على القيم المفقودة في test
missing_idx = test[test['review_score'].isna()].index

# تعويض القيم المفقودة في test باستخدام distribution train
test.loc[missing_idx, 'review_score'] = np.random.choice(choices, size=len(missing_idx), p=probs)


In [ ]:
train["review_score"].isna().sum()

In [ ]:
train["review_score"].value_counts()

### Summary
- adjust all values to be a score out of 5
- handle values % and /10
- imputed with random choice 

## gross_weight, net_weight, Package_weight columns

problems:

*   random strings
*   negative numbers
*   mixed data types (float, int, object)
*   mixed measuring units (grams, kg, oz, lbs)



In [ ]:
cols = ["gross_weight", "net_weight", 'package_weight']

for col in cols:
    print(f"Unique values in {col}:")
    print(train[col].unique()[:50])
    print("-" * 50)


### clean

In [ ]:
def extract_numbers_in_kg(x):
    if pd.isna(x):
        return np.nan

    s = str(x).strip().lower()

    # شيل التكست الزيادة
    s = re.sub(r"(amount:|val:|data:|measure:)", "", s).strip()

    # استخرج الرقم
    match = re.search(r"[-+]?\d*\.?\d+", s)
    if not match:
        return np.nan
    value = abs(float(match.group()))  # خلي الرقم موجب

    # تحويل للوحدة kg
    if "kg" in s:
        return value
    if "grams" in s or "gram" in s or re.search(r"\bg\b", s):
        return value / 1000
    if "lbs" in s or "lb" in s:
        return value * 0.453592
    if "oz" in s:
        return value * 0.0283495
    else:
        return value   # نفترض أنه kg لو مفيش وحدة واضحة

# الأعمدة المستهدفة
weights = ["gross_weight", "net_weight", 'package_weight']

for col in weights:
    train[col] = train[col].apply(extract_numbers_in_kg)
    train[col] = train[col].abs()

# تأكد من النتيجة
print(train[weights].head())


In [ ]:
train[weights]

In [ ]:
cols = ["gross_weight", "net_weight", 'package_weight']

for col in cols:
    print(f"{col} → {train[col].isna().sum()} missing values")

In [ ]:

def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# اكتشاف الأوتلايرز
gross_outliers = detect_outliers_iqr(train, "gross_weight")
net_outliers = detect_outliers_iqr(train, "net_weight")
package_outliers = detect_outliers_iqr(train, "package_weight")

# عرض الرسومات جنب بعض
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
sns.histplot(gross_outliers["gross_weight"], bins=30, kde=True)
plt.title("Gross Weight Outliers")

plt.subplot(1,3,2)
sns.histplot(net_outliers["net_weight"], bins=30, kde=True)
plt.title("Net Weight Outliers")

plt.subplot(1,3,3)
sns.histplot(package_outliers["package_weight"], bins=30, kde=True)
plt.title("Package Weight Outliers")

plt.tight_layout()
plt.show()



In [ ]:
cols = ["gross_weight", "net_weight", 'package_weight']
train[cols].isna().sum()

In [ ]:
train['package_weight'].describe()

In [ ]:
def fill_package_with_range(df, gross_outliers, net_outliers):

    df = df.copy()

    # Define valid range of package weights from data
    min_pkg = df["package_weight"].min()
    max_pkg = df["package_weight"].max()

    bad_gross = set(gross_outliers.index)
    bad_net = set(net_outliers.index)

    # Mask for safe filling
    mask = (
        df["package_weight"].isna()
        & df["gross_weight"].notna()
        & df["net_weight"].notna()
        & ~df.index.isin(bad_gross)
        & ~df.index.isin(bad_net)
    )

    # Candidate values
    candidate = df.loc[mask, "gross_weight"] - df.loc[mask, "net_weight"]

    # Keep only values inside the permitted range
    valid_mask = (candidate >= 0.1) & (candidate <= 5)

    # Assign only valid ones
    df.loc[mask[mask].index[valid_mask], "package_weight"] = candidate[valid_mask]

    return df
train = fill_package_with_range(train, gross_outliers, net_outliers)


In [ ]:
import re
import numpy as np
import pandas as pd

def extract_numbers_in_g(x):
    if pd.isna(x):
        return np.nan

    s = str(x).strip().lower()

    # شيل التكست الزيادة
    s = re.sub(r"(amount:|val:|data:|measure:)", "", s).strip()

    # استخرج الرقم
    match = re.search(r"[-+]?\d*\.?\d+", s)
    if not match:
        return np.nan
    value = abs(float(match.group()))  # خلي الرقم موجب

    # تحويل للوحدة gram
    if "kg" in s:
        return value * 1000
    if "grams" in s or "gram" in s or re.search(r"\bg\b", s):
        return value
    if "lbs" in s or "lb" in s:
        return value * 453.592
    if "oz" in s:
        return value * 28.3495
    else:
        return value  # نفترض أنه gram لو مفيش وحدة واضحة

# الأعمدة المستهدفة
weights = ["gross_weight", "net_weight", 'package_weight']

for col in weights:
    test[col] = test[col].apply(extract_numbers_in_g)
    test[col] = test[col].abs()

In [ ]:
cols = ["gross_weight", "net_weight", 'package_weight']
train[cols].isna().sum()

In [ ]:
def impute_package_by_group(df):
    df = df.copy()

    # Step 1: Group by department + brand (or product if detailed enough)
    group_median = df.groupby(["department", "brand"])["package_weight"].transform("median")

    # Step 2: Fill NaN with group median
    df["package_weight"] = df["package_weight"].fillna(group_median)

    # Step 3: Fallback - if still NaN, use department median
    dept_median = df.groupby("department")["package_weight"].transform("median")
    df["package_weight"] = df["package_weight"].fillna(dept_median)

    # Step 4: Fallback to global median if still NaN
    df["package_weight"] = df["package_weight"].fillna(df["package_weight"].median())

    return df
train = impute_package_by_group(train)


In [ ]:
def impute_package_by_group_test(test_df, train_df):
    test_df = test_df.copy()

    # Step 1: Group median from train
    group_median = train_df.groupby(["department", "brand"])["package_weight"].median()

    # Map median to test rows
    test_df["package_weight"] = test_df.apply(
        lambda row: group_median.get((row["department"], row["brand"]), np.nan), axis=1
    )

    # Step 2: Department median fallback from train
    dept_median = train_df.groupby("department")["package_weight"].median()
    test_df["package_weight"] = test_df["package_weight"].fillna(test_df["department"].map(dept_median))

    # Step 3: Global median fallback from train
    global_median = train_df["package_weight"].median()
    test_df["package_weight"] = test_df["package_weight"].fillna(global_median)

    return test_df

test = impute_package_by_group_test(test, train)


In [ ]:
cols = ["gross_weight", "net_weight", 'package_weight']
train[cols].isna().sum()

In [ ]:
train.loc[train["gross_weight"] < train["net_weight"], ["gross_weight",'net_weight', 'package_weight']]

In [ ]:
def impute_gross_net(df, gross_outliers, net_outliers):
    df = df.copy()

    # Extract indices from DataFrames
    gross_outliers = gross_outliers.index
    net_outliers = net_outliers.index

    # Mark bad values (NaN or outlier)
    bad_gross = df["gross_weight"].isna() | df.index.isin(gross_outliers)
    bad_net = df["net_weight"].isna() | df.index.isin(net_outliers)

    # Fill gross if it's bad but net is valid
    mask = bad_gross & (~bad_net)
    df.loc[mask, "gross_weight"] = df.loc[mask, "net_weight"] + df.loc[mask, "package_weight"]

    # Fill net if it's bad but gross is valid
    mask = bad_net & (~bad_gross)
    df.loc[mask, "net_weight"] = df.loc[mask, "gross_weight"] - df.loc[mask, "package_weight"]

    return df

train = impute_gross_net(train, gross_outliers, net_outliers)



In [ ]:
# افترض إن detect_outliers_iqr موجودة زي ما في train
test_gross_outliers = detect_outliers_iqr(test, "gross_weight")
test_net_outliers   = detect_outliers_iqr(test, "net_weight")


In [ ]:
# نفترض أنك حسبت outliers في test بنفس الطريقة اللي استخدمتها في train
# test_gross_outliers, test_net_outliers

def impute_gross_net_test(df, gross_outliers, net_outliers):
    df = df.copy()

    gross_outliers = gross_outliers.index
    net_outliers = net_outliers.index

    bad_gross = df["gross_weight"].isna() | df.index.isin(gross_outliers)
    bad_net = df["net_weight"].isna() | df.index.isin(net_outliers)

    # Fill gross if it's bad but net is valid
    mask = bad_gross & (~bad_net)
    df.loc[mask, "gross_weight"] = df.loc[mask, "net_weight"] + df.loc[mask, "package_weight"]

    # Fill net if it's bad but gross is valid
    mask = bad_net & (~bad_gross)
    df.loc[mask, "net_weight"] = df.loc[mask, "gross_weight"] - df.loc[mask, "package_weight"]

    return df

test = impute_gross_net_test(test, test_gross_outliers, test_net_outliers)


In [ ]:
cols = ["gross_weight", "net_weight", 'package_weight']
train[cols].isna().sum()

In [ ]:
train.loc[train["gross_weight"] < train["net_weight"], ["gross_weight",'net_weight', 'package_weight']]

In [ ]:
train.loc[train["gross_weight"] < train["net_weight"], "gross_weight"] = train["net_weight"] + train["package_weight"]

In [ ]:
test.loc[test["gross_weight"] < test["net_weight"], "gross_weight"] = test["net_weight"] + test["package_weight"]


In [ ]:
gross_outliers = detect_outliers_iqr(train, "gross_weight")
net_outliers = detect_outliers_iqr(train, "net_weight")
package_outliers = detect_outliers_iqr(train, "package_weight")

In [ ]:
train[cols].quantile(0.99)

In [ ]:

train.loc[train['gross_weight'] > train['gross_weight'].quantile(0.99), "gross_weight"] = train['gross_weight'].quantile(0.99)
train.loc[train['net_weight'] > train['net_weight'].quantile(0.99), "net_weight"] = train['net_weight'].quantile(0.99)

In [ ]:
# احصل على quantiles من train
gross_99 = train['gross_weight'].quantile(0.99)
net_99 = train['net_weight'].quantile(0.99)

# طبق capping على test
test.loc[test['gross_weight'] > gross_99, 'gross_weight'] = gross_99
test.loc[test['net_weight'] > net_99, 'net_weight'] = net_99


In [ ]:
'''
لو القيمة ناقصة في gross_weight (مثلاً):

نشوف المنتج / القسم / البراند اللي تابع له.

ناخد متوسط الوزن المعروف في نفس الجروب.

نعوض بيه القيمة الناقصة
'''

cols = ["gross_weight", "net_weight"]

# نعوض القيم المفقودة بالميديان لكل Department
for col in cols:
    train[col] = train.groupby("department")[col].transform(
        lambda x: x.fillna(x.median())
    )

# لو لسه فيه Missing (مثلاً مفيش أي بيانات للقسم ده)
# ممكن نعيد نفس الفكرة على مستوى الـ brand
for col in cols:
    train[col] = train.groupby("brand")[col].transform(
        lambda x: x.fillna(x.median())
    )

# لو لسه فيه Missing (حالات نادرة)
# نعوض بالميديان الكلي للعمود
for col in cols:
    train[col] = train[col].fillna(train[col].median())

# نطبع عدد النواقص بعد التعويض
print(train[cols].isna().sum())

In [ ]:
cols = ["gross_weight", "net_weight"]

# حساب medians من train
dept_medians = train.groupby("department")[cols].median()
brand_medians = train.groupby("brand")[cols].median()
global_medians = train[cols].median()

# تطبيق التعويض على test
for col in cols:
    # Step 1: Department median
    test[col] = test.apply(
        lambda row: dept_medians.loc[row["department"], col]
        if pd.isna(row[col]) and row["department"] in dept_medians.index
        else row[col], axis=1
    )
    # Step 2: Brand median
    test[col] = test.apply(
        lambda row: brand_medians.loc[row["brand"], col]
        if pd.isna(row[col]) and row["brand"] in brand_medians.index
        else row[col], axis=1
    )
    # Step 3: Global median
    test[col] = test[col].fillna(global_medians[col])




### Summary
- extract  numbers and convert all wights to Kg
- according to this equation gross_weight = net_weight + package_weight will impute this columns but first 
- since Package Weight was only present in one of the four datasets, it contained a high percentage of missing values
- before applying this formula we checked for outliers excluded it out of this calculation to avoid skewness
- package weight doesn't have outliers , gross and net outliers is not logical so it must be handled as nan
- so we address pakage weight first from this formoula if the other 2 columns are missing impute with group median of department/ brand.
- now we are ready to impute missing and outliers in gross and net from the formula
- only if the 2 other columns were missing we applied group-based imputation again.
- Note: since package weight range is small, impute it first minimize the error when using the formula 

## cost column
problems:
*   random texts
*   wrong data type
*   currancy mix


In [ ]:
# عرض القيم الفريدة في العمود cost
unique_values = train["cost"].unique()
print(unique_values[:200])


### clean

In [ ]:
conversion_to_usd = {
    '€': 1.1,    # مثال: 1 EUR = 1.1 USD
    '£': 1.25,   # مثال: 1 GBP = 1.25 USD
    'r$': 0.2    # مثال: 1 BRL = 0.2 USD
}

def clean_and_convert_to_usd(x):
    if pd.isna(x):
        return np.nan

    s = str(x).strip().lower()

    # 1- تحديد العملة
    currency = None
    if '€' in s:
        currency = '€'
    elif '£' in s:
        currency = '£'
    elif 'r$' in s:
        currency = 'r$'
    # الدولار $ بيكون الافتراضي

    # 2- استخراج الرقم فقط
    match = re.search(r"[-+]?\d*\.?\d+", s)
    if not match:
        return np.nan
    value = float(match.group())

    # 3- تحويل للعملة بالدولار
    if currency is not None:
        value = value * conversion_to_usd[currency]

    # 4- تحويل السالب لموجب
    value = abs(value)

    return value


In [ ]:
train["cost"] = train["cost"].apply(clean_and_convert_to_usd)

In [ ]:
# تطبيق الدالة على test
test['yearly_income'] = test['yearly_income'].apply(clean_and_convert_to_usd)

In [ ]:
train["cost"].isna().sum()

### dropna (target)

In [ ]:
train.dropna(subset=['cost'], inplace=True)

In [ ]:
# عرض القيم الفريدة في العمود cost
unique_values = train["cost"].unique()
print(unique_values[:50])


### outliers

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=train['cost'])
plt.title('Boxplot of Cost')
plt.xlabel('Cost')
plt.show()


مفيش اوت لاير الحمد لله

In [ ]:
sns.histplot(net_outliers["cost"], bins=10, kde=True)
plt.title("Boxplot of Cost")

### Summary
- extract the number and convert all to USD
- dropped missing data

## additinal_features_in_market
problems:
- it contain lists but they are in string format
- need splitting into columns

In [ ]:
train["additional_features_in_market"].unique()[:50]

### clean

In [ ]:
# unique values in this column
features = ["video store", "florist", "ready food", "coffee bar", "bar for salad"]

def parse_features(val):
    if pd.isna(val):
        return []
    # Remove extra characters and standardize
    val = str(val).lower().strip(" .")
    val = val.replace("(", "").replace(")", "")
    try:
        # Try safe literal_eval (for values like "['coffee bar', 'florist']")
        parsed = ast.literal_eval(val)
        if isinstance(parsed, list):
            return [f.strip().lower() for f in parsed]
    except:
        pass

    # If it's not a proper list, fallback: just split by comma
    return [x.strip().lower() for x in val.replace("[","").replace("]","").replace("'","").split(",") if x]

In [ ]:
train["additional_features_in_market"] = train["additional_features_in_market"].apply(parse_features)

In [ ]:
test["features_parsed"] = test["additional_features_in_market"].apply(parse_features)

In [ ]:
# creat feature columns
for f in features:
   train[f] = train["additional_features_in_market"].apply(lambda lst: 1 if f in lst else 0)

In [ ]:
# أولاً: parse features
test["features_parsed"] = test["additional_features_in_market"].apply(parse_features)

# بعدين: إنشاء الأعمدة الثنائية
for f in features:
    test[f] = test["features_parsed"].apply(lambda lst: 1 if f in lst else 0)


In [ ]:
train['num_features'] = train[features].sum(axis=1)

In [ ]:
test['num_features'] = test[features].sum(axis=1)

### impute

We don't need to because we will make drops for this col. As for the colms that came out, they are compensated by 0 if there is none.

In [ ]:
train['num_features'].isna().sum()

In [ ]:
train.drop(columns="additional_features_in_market", inplace =True)

In [ ]:
test.drop(columns="additional_features_in_market", inplace =True)

### Summary
- extract new columns with the unique values (video store, florist, ready food, coffee bar, bar for salad)
- present feature is set to 1 and absent set to 0
- extract new column: number of features
- dropped the original column

## Areas (Store, Grocery, Frozen, Meat) & stor kind
problems:
- nan values in store can be substituted by sum of the remaining areas
- there data types are object


In [ ]:
area = ['grocery_area', 'frozen_area', 'meat_area', 'store_area']

for col in area:
    # الحصول على القيم الفريدة، يشمل NaN
    unique_vals = train[col].unique()

    print(f"Unique values in '{col}' (count: {len(unique_vals)}):")
    print(unique_vals)
    print("-" * 40)


In [ ]:
# الفانكشن لاستخراج الرقم وتحويل السالب لموجب والتقريب
def extract_and_clean_number(text):
    if pd.isna(text):
        return np.nan

    match = re.search(r"[-+]?\d*\.?\d+", str(text))
    if not match:
        return np.nan

    num = float(match.group())
    num = abs(num)
    num = round(num, 2)
    return num

# الأعمدة اللي عايزين ننظفها
area = ['grocery_area', 'frozen_area', 'meat_area', 'store_area']

# تطبيق الفانكشن على كل عمود
for col in area:
    train[col] = train[col].apply(extract_and_clean_number)

# عرض أول صفوف للتأكد
print(train[area].head())


In [ ]:
# الفانكشن لاستخراج الرقم وتحويل السالب لموجب والتقريب
def extract_and_clean_number(text):
    if pd.isna(text):
        return np.nan

    match = re.search(r"[-+]?\d*\.?\d+", str(text))
    if not match:
        return np.nan

    num = float(match.group())
    num = abs(num)
    num = round(num, 2)
    return num

# الأعمدة اللي عايزين ننظفها
area = ['grocery_area', 'frozen_area', 'meat_area', 'store_area']

# تطبيق الفانكشن على كل عمود في test
for col in area:
    test[col] = test[col].apply(extract_and_clean_number)



In [ ]:
area_cols = ["grocery_area", "frozen_area", "meat_area"]

def clean_area_values(val):
    if pd.isna(val):
        return np.nan

    val = str(val).strip()  # نشيل المسافات

    if val in ["", '""']:   # قيم فاضية
        return np.nan

    # لو بين أقواس (معناها غالبًا سالب)
    is_negative = False
    if val.startswith("(") and val.endswith(")"):
        is_negative = True
        val = val[1:-1]  # نشيل الأقواس

    # نشيل أي علامات زيادة
    val = val.replace('"', '').replace("'", "").replace("$", "").strip(".")

    try:
        num = float(val)
        if is_negative:
            num = -num
        return num
    except:
        return np.nan

# تطبيق الفانكشن على كل الأعمدة
for col in area_cols:
    train[col] = train[col].apply(clean_area_values)


In [ ]:


# تطبيق الفانكشن على كل الأعمدة في test
for col in area_cols:
    test[col] = test[col].apply(clean_area_values)


### outliers impute "grocery_area", "frozen_area", "meat_area"

In [ ]:

area_cols = ["grocery_area", "frozen_area", "meat_area"]
plt.figure(figsize=(15,5))

for i, col in enumerate(area_cols, 1):
    plt.subplot(1, 3, i)
    sns.boxplot(y=train[col])
    plt.title(f"Boxplot of {col}")

plt.tight_layout()
plt.show()


In [ ]:
area_cols = ["grocery_area", "frozen_area", "meat_area"]

# حساب Bounds لكل عمود
bounds = {}
for col in area_cols:
    Q1 = train[col].quantile(0.25)
    Q3 = train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    bounds[col] = (lower, upper)

# حساب الميديان لكل فئة
medians_per_kind = {col: train.groupby("store_kind")[col].median() for col in area_cols}

def replace_outliers(row):
    for col in area_cols:
        if pd.notna(row[col]):
            lower, upper = bounds[col]

            # تحت اللوور باوند
            if row[col] < lower:
                if row['store_kind'] in ['small', 'unknown']:
                    continue
                else:
                    row[col] = medians_per_kind[col].get(row['store_kind'], row[col])

            # فوق الأبر باوند
            elif row[col] > upper:
                if row['store_kind'] == 'premium':
                    continue
                else:
                    row[col] = medians_per_kind[col].get(row['store_kind'], row[col])
    return row

# تطبيق
train = train.apply(replace_outliers, axis=1)

# التأكد
print(train[area_cols + ['store_kind']].head(10))


In [ ]:
area_cols = ["grocery_area", "frozen_area", "meat_area"]

# الحسابات من train (تم عملها بالفعل)
# bounds: dictionary with (lower, upper) per column
# medians_per_kind: dictionary with median per store_kind per column

def replace_outliers_test(row):
    for col in area_cols:
        if pd.notna(row[col]):
            lower, upper = bounds[col]

            # تحت اللوور باوند
            if row[col] < lower:
                if row['store_kind'] in ['small', 'unknown']:
                    continue
                else:
                    row[col] = medians_per_kind[col].get(row['store_kind'], row[col])

            # فوق الأبر باوند
            elif row[col] > upper:
                if row['store_kind'] == 'premium':
                    continue
                else:
                    row[col] = medians_per_kind[col].get(row['store_kind'], row[col])
    return row

# تطبيق على test
test = test.apply(replace_outliers_test, axis=1)



In [ ]:

area_cols = ["grocery_area", "frozen_area", "meat_area"]
plt.figure(figsize=(15,5))

for i, col in enumerate(area_cols, 1):
    plt.subplot(1, 3, i)
    sns.boxplot(y=train[col])
    plt.title(f"Boxplot of {col}")

plt.tight_layout()
plt.show()

In [ ]:
area = ['grocery_area', 'frozen_area', 'meat_area', 'store_area']

for col in area:
    # تحويل العمود للـ float
    train[col] = train[col].astype(float)

    # التأكد من النوع
    print(f"{col} dtype: {train[col].dtype}")

In [ ]:
area = ['grocery_area', 'frozen_area', 'meat_area', 'store_area']

for col in area:
    # تحويل العمود للـ float
    test[col] = test[col].astype(float)


### impute store_area

In [ ]:
train[area].isna().sum()

In [ ]:
area_cols = ['grocery_area', 'frozen_area', 'meat_area']

# تعويض NaN في store_area بمجموع الأعمدة الثلاثة
train['store_area'] = train.apply(
    lambda row: row[area_cols].sum() if pd.isna(row['store_area']) else row['store_area'], axis=1
)


In [ ]:
area_cols = ['grocery_area', 'frozen_area', 'meat_area']

# تعويض NaN في store_area بمجموع الأعمدة الثلاثة
test['store_area'] = test.apply(
    lambda row: row[area_cols].sum() if pd.isna(row['store_area']) else row['store_area'], axis=1
)


In [ ]:
train[area].isna().sum()

In [ ]:
converted = pd.to_numeric(train['grocery_area'], errors="coerce")

# Find the rows that failed to convert
failed = train.loc[converted.isna() & train['grocery_area'].notna(), col]
failed.unique()

In [ ]:
converted = pd.to_numeric(test['grocery_area'], errors="coerce")

# العثور على الصفوف اللي فشل تحويلها لرقم
failed = test.loc[converted.isna() & test['grocery_area'].notna(), 'grocery_area']
failed.unique()


### outliers

In [ ]:

plt.figure(figsize=(8, 6))
sns.boxplot(y=train['store_area'])
plt.title("Box Plot of Store Area")
plt.ylabel("Store Area")
plt.show()


In [ ]:
# حساب المتوسط مع تجاهل أي NaN
mean_store_area = train['store_area'].mean()

# استبدال القيم الأكبر من 4000 بالمتوسط
train.loc[train['store_area'] > 4000, 'store_area'] = mean_store_area

In [ ]:
# استبدال القيم الأكبر من 4000 بالمتوسط من train
test.loc[test['store_area'] > 4000, 'store_area'] = mean_store_area


In [ ]:

plt.figure(figsize=(8, 6))
sns.boxplot(y=train['store_area'])
plt.title("Box Plot of Store Area")
plt.ylabel("Store Area")
plt.show()


In [ ]:
# عدد القيم الأقل من 1000
count_below_1000 = (train['store_area'] < 1000).sum()

print(f"{count_below_1000}")


In [ ]:
# حساب الميديان لكل فئة في store_kind
median_per_kind = train.groupby('store_kind')['store_area'].median()

# دالة الاستبدال
def replace_small_store_area(row):
    if row['store_area'] < 1000:
        if row['store_kind'] in ['small', 'unknown']:
            return row['store_area']  # خليها زي ما هي
        else:
            # استبدلها بالميديان الخاص بالفئة اللي هي فيها
            return median_per_kind.get(row['store_kind'], row['store_area'])
    else:
        return row['store_area']     # القيم >= 1000 خليها زي ما هي

# تطبيق الدالة
train['store_area'] = train.apply(replace_small_store_area, axis=1)

# التأكد من النتائج
print(train[['store_area', 'store_kind']].head(10))



In [ ]:
# دالة الاستبدال للقيم الصغيرة في test
def replace_small_store_area_test(row):
    if row['store_area'] < 1000:
        if row['store_kind'] in ['small', 'unknown']:
            return row['store_area']  # خليها زي ما هي
        else:
            # استبدلها بالميديان الخاص بالفئة من train
            return median_per_kind.get(row['store_kind'], row['store_area'])
    else:
        return row['store_area']     # القيم >= 1000 خليها زي ما هي

# تطبيق الدالة على test
test['store_area'] = test.apply(replace_small_store_area_test, axis=1)



In [ ]:

plt.figure(figsize=(8, 6))
sns.boxplot(y=train['store_area'])
plt.title("Box Plot of Store Area")
plt.ylabel("Store Area")
plt.show()


In [ ]:
# عدد القيم الأقل من 1000
count_below_1000 = (train['store_area'] < 1000).sum()

print(f" {count_below_1000}")


So these are definitely not outlayers because they are present as a small area in the story kind column.

In [ ]:
print("Before:")
print(train['store_kind'].value_counts())

# حساب المتوسط لكل فئة بدون unknown
mean_per_category = train.loc[train['store_kind'] != 'unknown'].groupby('store_kind')['store_area'].mean()

# استبدال unknown بناءً على أقرب متوسط
def replace_unknown(row):
    if row['store_kind'] == 'unknown':
        # احسب الفروق مع كل فئة حقيقية
        diffs = {cat: abs(row['store_area'] - mean) for cat, mean in mean_per_category.items()}
        # اختار الفئة الأقرب
        closest_category = min(diffs, key=diffs.get)
        return closest_category
    else:
        return row['store_kind']

train['store_kind'] = train.apply(replace_unknown, axis=1)

# التأكد من النتائج
print("After:")
print(train['store_kind'].value_counts())



In [ ]:


# دالة الاستبدال للقيم unknown
def replace_unknown_test(row):
    if row['store_kind'] == 'unknown':
        # احسب الفروق مع كل فئة حقيقية من train
        diffs = {cat: abs(row['store_area'] - mean) for cat, mean in mean_per_category.items()}
        # اختار الفئة الأقرب
        closest_category = min(diffs, key=diffs.get)
        return closest_category
    else:
        return row['store_kind']

test['store_kind'] = test.apply(replace_unknown_test, axis=1)



problems
- remove () and . and ""

- store area = 	grocery_area + frozen_area + meat_area

### impute "grocery_area", "frozen_area", "meat_area"

In [ ]:

def fill_missing_areas(row):
    g, f, m, s = row["grocery_area"], row["frozen_area"], row["meat_area"], row["store_area"]
    missing = [pd.isna(g), pd.isna(f), pd.isna(m)]
    n_missing = sum(missing)

    if n_missing == 1:
        # حالة ناقص واحد
        if pd.isna(g):
            row["grocery_area"] = s - (f + m)
        elif pd.isna(f):
            row["frozen_area"] = s - (g + m)
        elif pd.isna(m):
            row["meat_area"] = s - (g + f)

    elif n_missing == 2:
        # وزع الفرق على الاتنين الناقصين
        known = g if not pd.isna(g) else (f if not pd.isna(f) else m)
        missing_value = (s - known) / 2
        if pd.isna(g): row["grocery_area"] = missing_value
        if pd.isna(f): row["frozen_area"] = missing_value
        if pd.isna(m): row["meat_area"] = missing_value

    elif n_missing == 3:
        # وزع المساحة كلها بالتساوي
        row["grocery_area"] = row["frozen_area"] = row["meat_area"] = s / 3

    return row

# تطبيق الدالة على tran
train = train.apply(fill_missing_areas, axis=1)


In [ ]:
def fill_missing_areas_test(row):
    g, f, m, s = row["grocery_area"], row["frozen_area"], row["meat_area"], row["store_area"]
    missing = [pd.isna(g), pd.isna(f), pd.isna(m)]
    n_missing = sum(missing)

    if n_missing == 1:
        # حالة ناقص واحد
        if pd.isna(g):
            row["grocery_area"] = s - (f + m)
        elif pd.isna(f):
            row["frozen_area"] = s - (g + m)
        elif pd.isna(m):
            row["meat_area"] = s - (g + f)

    elif n_missing == 2:
        # وزع الفرق على الاتنين الناقصين
        known = g if not pd.isna(g) else (f if not pd.isna(f) else m)
        missing_value = (s - known) / 2
        if pd.isna(g): row["grocery_area"] = missing_value
        if pd.isna(f): row["frozen_area"] = missing_value
        if pd.isna(m): row["meat_area"] = missing_value

    elif n_missing == 3:
        # وزع المساحة كلها بالتساوي
        row["grocery_area"] = row["frozen_area"] = row["meat_area"] = s / 3

    return row

# تطبيق الدالة على test
test = test.apply(fill_missing_areas_test, axis=1)


In [ ]:
area_cols = ["grocery_area", "frozen_area", "meat_area", "store_area"]
train[area_cols].describe()

In [ ]:
train[['grocery_area', 'frozen_area', 'meat_area', 'store_area']].isna().sum()

### Summary
- extract the numbers and handle negative values to be positive
- we will be using the formula to impute this column: store_area = grocery_area + frozen_area + meat_area
- impute outliers with the store kind group median
- now we can impute this columns with the formula 

## Store Cost and Sales
- Various Iconsistent Values
- Various Incorrect Values

In [ ]:
columns = ['store_sales', 'store_cost']

for col in columns:
    unique_vals = train[col].unique()[:50]
    print(f"Unique values in '{col}':")
    print(unique_vals)
    print("\n")  # سطر فاصل بين الأعمدة


### clean

In [ ]:
train[['store_sales', 'store_cost']].isna().sum()

In [ ]:
# same function used in the cost
train['store_sales'] = train['store_sales'].apply(clean_and_convert_to_usd)
train['store_cost'] = train['store_cost'].apply(clean_and_convert_to_usd)

In [ ]:
# same function used in the cost
test['store_sales'] = test['store_sales'].apply(clean_and_convert_to_usd)
test['store_cost'] = test['store_cost'].apply(clean_and_convert_to_usd)

In [ ]:
train[['store_sales', 'store_cost']].isna().sum()

### impute

In [ ]:
# الأعمدة المراد تعويضها
cols_to_fill = ['store_sales', 'store_cost']

for col in cols_to_fill:
    # نحسب المتوسط لكل store_kind
    mean_per_kind = train.groupby('store_kind')[col].transform('mean')

    # استبدال النال بالقيمة المتوسطة حسب الكاتيجوري
    train[col] = train[col].fillna(mean_per_kind)

# التأكد من أنه لا يوجد NaN بعد العملية
print(train[cols_to_fill].isna().sum())


In [ ]:
cols_to_fill = ['store_sales', 'store_cost']

for col in cols_to_fill:
    # حساب المتوسط لكل store_kind من train
    mean_per_kind = train.groupby('store_kind')[col].mean()

    # استبدال القيم المفقودة في test بالقيمة المتوسطة حسب الكاتيجوري
    test[col] = test.apply(
        lambda row: mean_per_kind.get(row['store_kind'], row[col]) if pd.isna(row[col]) else row[col], axis=1
    )

### outliers

In [ ]:
# دالة لاكتشاف الأوتلاير باستخدام IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# اكتشاف الأوتلايرز
sales_outliers = detect_outliers_iqr(train, "store_sales")
cost_outliers = detect_outliers_iqr(train, "store_cost")

# عرض الرسومات جنب بعض
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
sns.boxplot(y=train["store_sales"])
plt.title("Store Sales Outliers")

plt.subplot(1,2,2)
sns.boxplot(y=train["store_cost"])
plt.title("Store Cost Outliers")

plt.tight_layout()
plt.show()



In [ ]:
# شرط القيم الأكبر من 15 مليون وخارج فئة premium
condition_sales = (train['store_sales'] > 15_000_000) & (train['store_kind'] != 'premium')
condition_cost  = (train['store_cost']  > 15_000_000) & (train['store_kind'] != 'premium')

# حساب عدد الصفوف لكل عمود
count_sales = train[condition_sales].shape[0]
count_cost  = train[condition_cost].shape[0]

print(f"Number of store_sales > 15 million and not in 'premium': {count_sales}")
print(f"Number of store_cost > 15 million and not in 'premium': {count_cost}")



All the extreme values ​​are in very large stores like malls, so it is normal for the profit to be this high, so this is not an outlier or anything.

### Summary
- extract numbers and convert all currency to USD
- impute missing values with the mean cost/sales of the store kind (apply groupby)
- kept the outliers, as they were determined to be logically valid.

_____

# EDA & Visualization

### Correlation Heatmap Insights

1. **Strong correlation between `gross_weight` and `net_weight`** (≈ 0.97)  
   - Expected, since gross weight includes packaging, while net weight is the product only.

2. **Strong correlation between `store_sales` and `store_cost`** (≈ 0.95)  
   - Makes sense: higher sales usually imply higher costs.

3. **`store_area` is moderately correlated with `grocery_area`, `frozen_area`, and `meat_area`** (0.3–0.4)  
   - Larger stores tend to have larger dedicated sections.

4. **Features like `video store`, `florist`, `ready food`, `coffee bar`, `bar for salad`, and `num_features` are strongly correlated with each other** (0.5–0.8)  
   - These form a **services/features cluster**, meaning that the presence of one feature increases the likelihood of others.

5. **`cost` has weak correlations with almost all numeric features**  
   - Suggests that cost may be more influenced by **categorical factors** such as `promotion_name` or `brand`.

---
## Categorical Variables

1. **Strong Associations**
   - **Product ↔ Department (0.82):** Very strong association, which is expected since departments categorize products.
   - **Department ↔ Brand (0.53):** Brands tend to cluster within specific departments.
   - **Product ↔ Brand (0.40):** Moderate-to-strong link, reflecting that brands are tied to particular products.
   - **Education ↔ Work (0.50):** Education level strongly influences work category/type.

2. **Moderate Associations**
   - **Promotion Name ↔ Store Kind (0.24):** Certain promotions are associated with specific store categories.

---
### Categorical vs Numerical Analysis

1. Store kind shows very strong associations with several numerical features, especially the total number of features and ready food sales. Salad bars, florists, and coffee bars. This indicates that the type of store strongly influences the range of services and specific departments offered.  

2. Work is highly related to yearly income, which aligns with expectations. Education also shows a meaningful connection with income, reflecting how academic level influences earning potential.  

3. Promotion name is moderately associated with cost, store size, grocery area, and certain store sections such as coffee bar, salad bar, florist, and ready food. This suggests that promotions are not random but tailored to store setup and product types.  

---
## Notes on Cost Relationships

- **Promotion Name** shows the strongest relationship with `cost`
- Other features like *bar for salad*, *ready food*, *florist*, and *coffee bar* are statistically significant , but their actual correlations are weak
- From a **practical perspective**, only `promotion_name` seems to have a meaningful influence on `cost`.  
- Most other variables show negligible impact and can be deprioritized in further analysis.

In [ ]:
train.shape
train.info()

In [ ]:
train.head()

I whispered every time I saw how important it was with the cost, because if it was important, I would drop it.

## All Columns

In [ ]:
print("=== Correlation Matrix ===")
numeric_cols = train.select_dtypes(include=["int64", "float64"]).columns
corr = train[numeric_cols].corr()

# Heatmap of correlations
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

- Cost depends on the store area and the number of features.
- Of course, there is a strong relationship between sales and cost.
- All weights have a strong relationship.

In [ ]:
def eda_summary(df, top_rare=3):
    summary = pd.DataFrame({
        "Data_Type": df.dtypes,
        "Missing_Values": df.isna().sum(),
        "Missing_Percent": round(df.isna().mean() * 100, 1),
        "Unique_Values": df.nunique(),
        "Maximum_Value": df.max(),
        "Minimum_Value": df.min(),
    })

    numerical_cols = df.select_dtypes(include=np.number).columns
    outliers_values = {}
    outliers_count = {}
    for col in numerical_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
        outliers_values[col] = outliers.tolist()
        outliers_count[col] = len(outliers)

    summary["Outliers_Count"] = summary.index.map(lambda x: outliers_count.get(x, np.nan))
    summary["Outliers"] = summary.index.map(lambda x: outliers_values.get(x, np.nan))

    categorical_cols = df.select_dtypes(include='object').columns
    rare_values = {}
    for col in categorical_cols:
        counts = df[col].value_counts()
        rare_values[col] = counts.nsmallest(top_rare).to_dict()
    summary["Rare_Values"] = summary.index.map(lambda x: rare_values.get(x, np.nan))

    return summary.sort_values(by="Missing_Values", ascending=False)

In [ ]:
from scipy.stats import f_oneway
# correlation ratio function (eta squared)
def correlation_ratio(categories, values):
    categories = np.array(categories)
    values = np.array(values)

    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat) + 1
    y_avg = np.nanmean(values)

    n = np.zeros(cat_num)
    y_cat = np.zeros(cat_num)

    for i in range(cat_num):
        cat_measures = values[np.argwhere(fcat == i).flatten()]
        n[i] = len(cat_measures)
        y_cat[i] = np.nanmean(cat_measures) if len(cat_measures) > 0 else 0

    numerator = np.sum(n * (y_cat - y_avg) ** 2)
    denominator = np.sum((values - y_avg) ** 2)

    return numerator / denominator if denominator != 0 else 0


def cost_relationships_grid(df, cost_col="cost", eta_threshold=0.05, p_threshold=0.05, corr_threshold=0.1, top_cats_limit=20):
    categorical_cols = df.select_dtypes(include="object").columns.tolist()
    numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
    if cost_col in numerical_cols:
        numerical_cols.remove(cost_col)

    plots = []

    # ----- Numerical vs Cost -----
    for num_col in numerical_cols:
        corr = df[[cost_col, num_col]].corr().iloc[0,1]
        if abs(corr) >= corr_threshold:
            plots.append(("num", num_col, corr, df))

    # ----- Categorical vs Cost -----
    for cat_col in categorical_cols:
        top_cats = df[cat_col].value_counts().nlargest(top_cats_limit).index
        df_plot = df.copy()
        df_plot[cat_col] = df_plot[cat_col].where(df_plot[cat_col].isin(top_cats), "Other")

        groups = [df_plot[df_plot[cat_col] == g][cost_col].dropna() for g in df_plot[cat_col].unique()]
        if len(groups) <= 1:
            continue

        eta = correlation_ratio(df_plot[cat_col].astype(str), df_plot[cost_col])
        try:
            _, p_value = f_oneway(*groups)
        except:
            p_value = 1

        if eta >= eta_threshold and p_value < p_threshold:
            plots.append(("cat", cat_col, eta, df_plot))

    if not plots:
        print("No significant relationships found.")
        return

    n = len(plots)
    cols = 3
    rows = int(np.ceil(n / cols))

    fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))
    axes = axes.flatten()

    for ax, (ptype, col, score, data_used) in zip(axes, plots):
        if ptype == "num":
            sns.scatterplot(x=col, y=cost_col, data=data_used, alpha=0.5, ax=ax)
            ax.set_title(f"{col} vs {cost_col}\nCorr={score:.2f}")
        else:
            sns.boxplot(x=col, y=cost_col, data=data_used, ax=ax)
            ax.set_title(f"{col} vs {cost_col}\nEta²={score:.2f}")
            ax.tick_params(axis='x', rotation=90)

    for j in range(len(plots), len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

In [ ]:
cost_relationships_grid(train, cost_col="cost")

## yearly_income

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(train['yearly_income'], train['cost'], alpha=0.5)

plt.title("Scatter Plot: Yearly Income vs Cost")
plt.xlabel("Yearly Income")
plt.ylabel("Cost")
plt.grid(True)
plt.show()

 - There's no clear relationship between an individual's annual income and the amount they spend on supermarket purchases.
- You'll feel like this column is divided into classes, so I'll divide it into categories in a new column.

In [ ]:

import pandas as pd

bins = [0, 40000, 80000, 120000, 200000]
labels = ['Low', 'Medium', 'High', 'Very High']

train['income_category'] = pd.cut(train['yearly_income'], bins=bins, labels=labels, include_lowest=True)



In [ ]:
bins = [0, 40000, 80000, 120000, 200000]
labels = ['Low', 'Medium', 'High', 'Very High']

test['income_category'] = pd.cut(test['yearly_income'], bins=bins, labels=labels, include_lowest=True)


## gross_weight, net_weight, package_weight

In [ ]:


# List of features to compare with cost
features = ['gross_weight', 'net_weight', 'package_weight']

plt.figure(figsize=(15, 5))

for i, col in enumerate(features, 1):
    plt.subplot(1, 3, i)
    plt.scatter(train[col], train['cost'], alpha=0.5)
    plt.xlabel(col)
    plt.ylabel('Cost')
    plt.title(f"{col} vs Cost")

plt.tight_layout()
plt.show()


- It makes sense for someone to buy something light and pay a high price for it, like a gold ring, for example.
- It's natural for there to be no clear relationship between weight and price.

## recyclable

In [ ]:

plt.figure(figsize=(10,5))

# Boxplot
plt.subplot(1,2,1)
sns.boxplot(x='is_recyclable', y='cost', data=train)
plt.title("Boxplot: Recycle vs Cost")

# Barplot (mean cost)
plt.subplot(1,2,2)
sns.barplot(x='is_recyclable', y='cost', data=train, estimator=lambda x: round(x.mean(),2))
plt.title("Mean Cost per Recycle Category")

plt.tight_layout()
plt.show()


- Colum has nothing to do with the cost.
- If the Yes were much higher than the No, or vice versa, I would have said this is an important feature.
- But neither of them is higher than the other, so this Colum is not expressive.

## promotion_name

In [ ]:
plt.figure(figsize=(15,6))
sns.barplot(x="promotion_name", y="cost", data=train, estimator=lambda x: round(x.mean(),2))
plt.xticks(rotation=90)
plt.title("Average Cost by Promotion")
plt.show()


- This column is useful in predicting cost because there are campaigns that attracted more customers than others.
- We won't delete it, but we'll target/freq encoding it.

## store_kind

In [ ]:
plt.figure(figsize=(14,6))

# Barplot
plt.subplot(1,2,1)
sns.barplot(x="store_kind", y="cost", data=train, estimator=lambda x: round(x.mean(),2))
plt.title("Average Cost by Store Kind")

# Violinplot
plt.subplot(1,2,2)
sns.violinplot(x="store_kind", y="cost", data=train)
plt.title("Cost Distribution by Store Kind")

plt.tight_layout()
plt.show()


- Unfortunately, there is no strong relationship between him and the cost.


## store_area

In [ ]:

# 1) Scatter plot
plt.figure(figsize=(8,5))
sns.scatterplot(x='store_area', y='cost', data=train, alpha=0.5)
plt.title("Scatter Plot: Store Area vs Cost")
plt.xlabel("Store Area")
plt.ylabel("Cost")
plt.show()

# 2) Boxplot باستخدام تقسيم المساحات إلى فئات (bins)
train['store_area_bins'] = pd.cut(train['store_area'], bins=[0,1000,2000,3000,4000,5000],
                                  labels=['<1k','1k-2k','2k-3k','3k-4k','4k+'])

plt.figure(figsize=(8,5))
sns.boxplot(x='store_area_bins', y='cost', data=train)
plt.title("Boxplot: Store Area (binned) vs Cost")
plt.xlabel("Store Area (bins)")
plt.ylabel("Cost")
plt.show()


- The Store Area is useful if we divide it into classes based on space.
- I will divide it into classes and dispense with the Store Kind.

In [ ]:
# إنشاء كولوم جديد للفئات
train['store_area_category'] = pd.cut(
    train['store_area'],
    bins=[0,1000,2000,3000,4000],
    labels=['<1k','1k-2k','2k-3k','3k-4k']
)

# نتاكد من النتيجة
print(train[['store_area', 'store_area_category']].head(10))


In [ ]:
# نطبق نفس الـ bins على test
bins = [0, 1000, 2000, 3000, 4000]
labels = ['<1k', '1k-2k', '2k-3k', '3k-4k']

test['store_area_category'] = pd.cut(
    test['store_area'],
    bins=bins,
    labels=labels
)

- Apply one hot encoding  to it.


## store_sales , store_cost

In [ ]:

plt.figure(figsize=(12,5))

# العلاقة بين store_sales و cost
plt.subplot(1,2,1)
sns.scatterplot(data=train, x="store_sales", y="cost", alpha=0.5)
plt.title("Store Sales vs Cost")

# العلاقة بين store_cost و cost
plt.subplot(1,2,2)
sns.scatterplot(data=train, x="store_cost", y="cost", alpha=0.5)
plt.title("Store Cost vs Cost")

plt.tight_layout()
plt.show()


- The heatmap above shows that the relationship between sales and the store coast is strong.
- In my opinion, we should remove the store coast and include the store sales model only, because the two are almost the same.

## customer_city, customer_state , seller_city, seller_state

In [ ]:


plt.figure(figsize=(18, 14))

# 1. Average cost by customer_state
plt.subplot(2, 2, 1)
sns.barplot(x='customer_state', y='cost', data=train, errorbar=None)
plt.title("Average Cost by Customer State", fontsize=12)
plt.xticks(rotation=90)

# 2. Average cost by seller_state
plt.subplot(2, 2, 2)
sns.barplot(x='seller_state', y='cost', data=train, errorbar=None)
plt.title("Average Cost by Seller State", fontsize=12)
plt.xticks(rotation=90)

# 3. Average cost by Top 10 customer cities
plt.subplot(2, 2, 3)
top_customer_cities = train['customer_city'].value_counts().nlargest(20).index
sns.barplot(x='customer_city', y='cost', data=train[train['customer_city'].isin(top_customer_cities)], errorbar=None)
plt.title("Average Cost by Top 20 Customer Cities", fontsize=12)
plt.xticks(rotation=45)

# 4. Average cost by Top 10 seller cities
plt.subplot(2, 2, 4)
top_seller_cities = train['seller_city'].value_counts().nlargest(20).index
sns.barplot(x='seller_city', y='cost', data=train[train['seller_city'].isin(top_seller_cities)], errorbar=None)
plt.title("Average Cost by Top 20 Seller Cities", fontsize=12)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


- All cities and states have very close cost ranges.
- I will do better frequency encoding for them than target encoding.

## review_score

In [ ]:

plt.figure(figsize=(8,5))
sns.barplot(x='review_score', y='cost', data=train, errorbar=None)
plt.title("Average Cost by Review Score", fontsize=14)
plt.show()


In [ ]:
# 2) حساب الميديان للكوست في كل كاتيجوري من الريفيو سكور
median_values = train.groupby("review_score")["cost"].median().round(2)
print("Median cost per review score:")
print(median_values)


- The review_score is not directly related to cost.
That is, the customer rating (1 to 5) reflects the customer experience (shipping delays, product quality, etc.), not the price.
A customer might purchase a very expensive order and receive a replacement if the service is poor.
- The data distribution is unbalanced.
The number of people who rated 1 is very small compared to those who rated 5, so the average can be misleading.

## gender, status, children ,education,	work

In [ ]:

categorical_cols = ['gender', 'status', 'children', 'education', 'work']

plt.figure(figsize=(18, 12))

for i, col in enumerate(categorical_cols, 1):
    plt.subplot(2, 3, i)
    sns.barplot(x=col, y='cost', data=train, estimator=lambda x: round(x.mean(), 2), errorbar=None)
    plt.title(f'Average Cost vs {col}')

plt.tight_layout()
plt.show()


In [ ]:


categorical_cols = ['gender', 'status', 'children', 'education', 'work']

plt.figure(figsize=(18, 12))

for i, col in enumerate(categorical_cols, 1):
    plt.subplot(2, 3, i)  # تقسيم الرسومات 2 صف × 3 أعمدة
    sns.boxplot(x=col, y='cost', data=train)
    plt.title(f'Cost vs {col}')

plt.tight_layout()
plt.show()


The only important things are the number of children and the work.

In [ ]:
categorical_cols = ['gender', 'status', 'children', 'education', 'work']
for col in categorical_cols :
  print(train[col].value_counts())

In [ ]:

# حساب المتوسط والإجمالي
gender_stats = train.groupby("gender")["cost"].agg(["mean", "sum"]).reset_index()

# رسم الأعمدة جنب بعض
fig, axes = plt.subplots(1, 2, figsize=(12,5))

# المتوسط
sns.barplot(data=gender_stats, x="gender", y="mean", ax=axes[0], palette="Set2")
axes[0].set_title("Mean for gender")
axes[0].set_ylabel("Mean Cost")

# الإجمالي
sns.barplot(data=gender_stats, x="gender", y="sum", ax=axes[1], palette="Set2")
axes[1].set_title("Sum for gender")
axes[1].set_ylabel("Total Cost")

plt.tight_layout()
plt.show()


- The sum of females is greater, but only because their number is greater in the data.
- The same thing will happen with singles and marrieds.
- I will keep all these columns as reserves

## product,	department,	brand

In [ ]:

# 1) department
mean_cost_department = train.groupby("department")["cost"].mean().sort_values(ascending=False)
plt.figure(figsize=(10,5))
sns.barplot(x=mean_cost_department.index, y=mean_cost_department.values)
plt.title("Average Cost per Department")
plt.ylabel("Average Cost")
plt.xlabel("Department")
plt.xticks(rotation=45)
plt.show()

# 2) brand (Top 10)
mean_cost_brand = train.groupby("brand")["cost"].mean().sort_values(ascending=False).head(20)
plt.figure(figsize=(12,6))
sns.barplot(x=mean_cost_brand.index, y=mean_cost_brand.values)
plt.title("Top 10 Brands by Average Cost")
plt.ylabel("Average Cost")
plt.xlabel("Brand")
plt.xticks(rotation=45)
plt.show()

# 3) product (Top 10)
mean_cost_product = train.groupby("product")["cost"].mean().sort_values(ascending=False).head(20)
plt.figure(figsize=(12,6))
sns.barplot(x=mean_cost_product.index, y=mean_cost_product.values)
plt.title("Top 10 Products by Average Cost")
plt.ylabel("Average Cost")
plt.xlabel("Product")
plt.xticks(rotation=45)
plt.show()



- I'll leave them, but I'll enter frequency encoding instead of target encoding, because target encoding in this case has no meaning.

## distance_km

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x="distance_km", y="cost", data=train, alpha=0.4)
sns.regplot(x="distance_km", y="cost", data=train, scatter=False, color="red")  # خط اتجاه

plt.title("Relationship between Distance (km) and Cost")
plt.xlabel("Distance (km)")
plt.ylabel("Cost")
plt.show()

- Colum has no effect on the cost

## num_features

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16,6))

# Boxplot
sns.boxplot(x="num_features", y="cost", data=train, ax=axes[0])
axes[0].set_title("Cost distribution by Number of Features")
axes[0].set_xlabel("Number of Features")
axes[0].set_ylabel("Cost")

# Barplot (mean values)
sns.barplot(x="num_features", y="cost", data=train, estimator=np.mean, ci=None, ax=axes[1])
axes[1].set_title("Average Cost by Number of Features")
axes[1].set_xlabel("Number of Features")
axes[1].set_ylabel("Average Cost")

plt.tight_layout()
plt.show()


- We will keep this column.

## Conclusion

In [ ]:
train.isna().sum()

In [ ]:
train = train.dropna()

In [ ]:
train.drop(columns='store_area_bins', inplace=True)

In [ ]:
train.columns

In [ ]:
cols_to_drop = ["is_recyclable", 'status', 'gender','review_score',
    "customer_city", "seller_city", "distance_km" ,"id"
]

train = train.drop(columns=cols_to_drop)

In [ ]:
cols_to_drop = ["is_recyclable", 'status', 'gender','review_score',
    "customer_city", "seller_city", "distance_km"
]
test = test.drop(columns=cols_to_drop)

In [ ]:
train.head()

# Preprocessing

In [ ]:
train.head()

- brand & promotion_name >> one hot encoding is better
- city & state >> target/freq encoder is better

In [ ]:
cols_to_encode = ["children", 'education',"work", "num_features", 'store_kind', 'income_category', 'store_area_category']

print("Shape Befor encoding:", train.shape)

# one hot encoding
train = pd.get_dummies(train, columns=cols_to_encode, drop_first=True)

# نتأكد إن أعمدة الـ one hot كلها 0/1 integer
dummy_cols = [col for col in train.columns if any(prefix in col for prefix in cols_to_encode)]
train[dummy_cols] = train[dummy_cols].astype(int)

print("Shape after encoding:", train.shape)
train.head()




In [ ]:
cols_to_encode = ["children", 'education',"work", "num_features", 'store_kind', 'income_category', 'store_area_category']

print("Shape before encoding:", test.shape)

# one hot encoding
test = pd.get_dummies(test, columns=cols_to_encode, drop_first=True)

# نتأكد إن أعمدة الـ one hot كلها 0/1 integer
dummy_cols = [col for col in test.columns if any(prefix in col for prefix in cols_to_encode)]
test[dummy_cols] = test[dummy_cols].astype(int)

# مزامنة الأعمدة مع train (لو في أعمدة ناقصة أو زيادة)
for col in train.columns:
    if col not in test.columns:
        test[col] = 0
test = test[train.columns]  # ترتيب الأعمدة زي train



In [ ]:
# =============================
#  Frequency Encoding
# =============================
cat_cols = ['seller_state', 'customer_state', 'promotion_name',
            'brand', 'product', 'department']

for col in cat_cols:
    freq_col = f"{col}_freq_enc"
    train[freq_col] = train[col].map(train[col].value_counts())

print("Frequency Encoding done:", cat_cols)


In [ ]:
cat_cols = ['seller_state', 'customer_state', 'promotion_name',
            'brand', 'product', 'department']

for col in cat_cols:
    freq_col = f"{col}_freq_enc"
    freq_mapping = train[col].value_counts()
    test[freq_col] = test[col].map(freq_mapping)

# لو فيه قيم جديدة في test مش موجودة في train → NaN نقدر نعوض بصفر
for col in cat_cols:
    freq_col = f"{col}_freq_enc"
    test[freq_col] = test[freq_col].fillna(0)

print("Frequency Encoding done for test:", cat_cols)


In [ ]:
train.drop(columns=cat_cols, inplace=True)

In [ ]:
test.drop(columns=cat_cols, inplace=True)

In [ ]:
train.shape

- Frequency encoding is better with Random Forest
- Scaling is prohibited as long as we work on Random Forest.

In [ ]:
X_train = train.drop("cost", axis=1)
y_train = train['cost']

In [ ]:
# from sklearn.model_selection import train_test_split

# # Use only 10% of the training data to test the models
# X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=0.90, random_state=42)

# Modeling

 Get the best parameters using only 10% of the data 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold

# Hyperparameter Grid

param_dist = {
    "n_estimators": [100, 200, 300, 500, 800, 1000],
    "max_depth": [None, 10, 20, 30, 40, 50, 60],
    "min_samples_split": [2, 5, 10, 15, 20],
    "min_samples_leaf": [1, 2, 4, 6, 8, 10],
    "max_features": ["sqrt", "log2", None]
}

rf = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    scoring="neg_root_mean_squared_error",
    random_state=42,
    n_jobs=-1
)

# تدريب RandomizedSearchCV على الـ Train set
random_search.fit(X_train, y_train)

print("Best Params:", random_search.best_params_)
print("Best CV Score (neg RMSE):", random_search.best_score_)

Train the final model with the whole dataset

In [ ]:
# Cross-validation Evaluation

best_rf = random_search.best_estimator_

kf = KFold(n_splits=5, shuffle=True, random_state=42)

rmse_scores = []
r2_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train), 1):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    best_rf.fit(X_tr, y_tr)
    preds = best_rf.predict(X_val)

    rmse = np.sqrt(((y_val - preds) ** 2).mean())
    r2 = 1 - ((y_val - preds) ** 2).sum() / ((y_val - y_val.mean()) ** 2).sum()

    rmse_scores.append(rmse)
    r2_scores.append(r2)

    print(f"Fold {fold} → CV RMSE: {rmse:.3f} | CV R²: {r2:.3f}")

print("\n--- Cross-validation Evaluation on Train Set ---")
print(f"Average CV RMSE: {np.mean(rmse_scores):.3f}")
print(f"Average CV R²: {np.mean(r2_scores):.3f}")

In [ ]:
X_test = test.copy()
X_test = X_test[X_train.columns]

In [ ]:
preds_test = best_rf.predict(X_test)
preds_test[:10] 

In [ ]:
submission = pd.DataFrame({
    "id": range(len(preds_test)),  
    "cost": preds_test           
})

print(submission.head())

In [ ]:
submission

In [ ]:
last_id = submission['id'].iloc[-1]
print(last_id)

In [ ]:
submission.to_csv("submission.csv", index=False)

print("File saved as submission.csv")

In [ ]:
# # Get feature importance
# #best_rf.fit(X_train, y_train)

# importances = pd.Series(best_rf.feature_importances_, index=X_train.columns)
# importances = importances.sort_values(ascending=False)

# # Show top 20
# plt.figure(figsize=(10,6))
# importances.head(50).plot(kind='bar')
# plt.title("Feature Importance (Random Forest)")
# plt.show()